In [1]:
from confluent_kafka import Producer
import json


In [ ]:
p = Producer({'bootstrap.servers': 'kafka-1:19092,kafka-2:19093'})


In [2]:
def delivery_report(err, msg):
    """ Called once for each message produced to indicate delivery result.
        Triggered by poll() or flush(). """
    if err is not None:
        print('Message delivery failed: {}'.format(err))
    else:
        print('Message delivered Timestamp:{timestamp} to {} [{}] key:{}'.format(msg.topic(), msg.partition(), msg.key(),timestamp=msg.timestamp()))


In [ ]:
lastone = ""

#def WritePM10(boxJsonString):
#    box = json.loads(boxJsonString)
#    
#    onlyNew= False
#    global lastone
#    lastRead = lastone
#    for s in box['sensors']:   
#      #  print("sensor tiles:{}".format(s['title']))
#        if s['title'] == "PM10":
#            isNewValue= not lastRead or (
#                lastRead['lastMeasurement']['createdAt'] != 
#                s['lastMeasurement']['createdAt'])
#            if(not onlyNew or isNewValue):
#              #  printTemperatureFromBox(s)
#                lastone = s                
#                p.poll(0)
#                # Asynchronously produce a message, the delivery report callback
#                # will be triggered from poll() above, or flush() below, when the message has
#                # been successfully delivered or failed permanently.
#                data =  s['lastMeasurement']['value']
#                p.produce("clean-latest", key=box["_id"], value = data.encode('utf-8'), callback=delivery_report)
#
#                # Wait for any outstanding messages to be delivered and delivery report
#                # callbacks to be triggered.
#                p.flush()
#

In [ ]:
#%pip install -U avro-python3

In [12]:

value_schema_str = """

"""

key_schema_str = """

"""


In [18]:
def ReadSchemafromFile() :
    result= ""
    with open('opensensemapBox.avsc') as json_file:
        schemaJson = json.load(json_file)
        result = json.dumps(schemaJson)
    return result

def ReadKeySchemafromFile() :
    result= ""
    with open('opensensemapBoxId.avsc') as json_file:
        schemaJson = json.load(json_file)
        result = json.dumps(schemaJson)
    return result



In [42]:
from confluent_kafka import avro
from confluent_kafka.avro import AvroProducer


value_schema = avro.loads(ReadSchemafromFile())
key_schema = avro.loads(ReadKeySchemafromFile())
avroProducer = AvroProducer({
    'bootstrap.servers': 'kafka-1:19092,kafka-1:19093',
    'schema.registry.url': 'http://schema-registry-1:8081',
    'compression.codec': 'snappy'
    }, default_key_schema=key_schema, default_value_schema=value_schema)

In [43]:

#value = {"id":"1001", "firstName": "Peter", "lastName": "Muster"}
#key = {"id": "1001"}
#avroProducer.produce(topic='test-avro-topic', value=value, key=key)
#avroProducer.flush()

In [44]:

def getValueAvro(box) :
    return boxValue

def getKey(box) :
    return boxKey


lastone = ""
def WritePM10Avro(boxJsonString):
    box = json.loads(boxJsonString)
    
    onlyNew= False
    global lastone
    lastRead = lastone
    isNewValue= not lastRead or (
                box['createdAt'] != box['createdAt'])
    if(not onlyNew or isNewValue):
        lastone = box
        #p.poll(0)
        value = box
        key = {"id": box["_id"]}
        avroProducer.produce(topic='"clean-latest"', value=value, key=key)
        avroProducer.flush()


In [45]:
# test
box = {'_id': '593fc68947df4f001113f9eb',
 'createdAt': '2017-06-13T11:03:37.041Z',
 'updatedAt': '2020-05-18T14:24:30.336Z',
 'name': 'Freiburg im Breisgau',
 'grouptag': 'XK',
 'exposure': 'outdoor',
 'model': 'luftdaten_sds011_dht22',
 'sensors': [{'title': 'PM10',
   'unit': 'µg/m³',
   'sensorType': 'SDS 011',
   'icon': 'osem-cloud',
   '_id': '593fc68947df4f001113f9ec',
   'lastMeasurement': {'value': '5.05',
    'createdAt': '2020-05-18T14:24:30.332Z'}},
  {'title': 'PM2.5',
   'unit': 'µg/m³',
   'sensorType': 'SDS 011',
   'icon': 'osem-cloud',
   '_id': '593fc68947df4f001113f9ed',
   'lastMeasurement': {'value': '4.75',
    'createdAt': '2020-05-18T14:24:30.332Z'}},
  {'title': 'Temperatur',
   'unit': '°C',
   'sensorType': 'DHT22',
   'icon': 'osem-thermometer',
   '_id': '593fc68947df4f001113f9ee',
   'lastMeasurement': {'value': '24.30',
    'createdAt': '2020-05-18T14:24:30.332Z'}},
  {'title': 'rel. Luftfeuchte',
   'unit': '%',
   'sensorType': 'DHT22',
   'icon': 'osem-humidity',
   '_id': '593fc68947df4f001113f9ef',
   'lastMeasurement': {'value': '76.40',
    'createdAt': '2020-05-18T14:24:30.332Z'}}],
 'currentLocation': {'coordinates': [7.854586, 47.965607],
  'type': 'Point',
  'timestamp': '2017-06-13T11:03:37.000Z'},
 'lastMeasurementAt': '2020-05-18T14:24:30.332Z',
 'loc': [{'geometry': {'coordinates': [7.854586, 47.965607],
    'type': 'Point',
    'timestamp': '2017-06-13T11:03:37.000Z'},
   'type': 'Feature'}]}




In [47]:
#WritePM10Avro(json.dumps(box))

In [49]:
topicName ='raw'
from confluent_kafka import Consumer, KafkaError

c = Consumer({
    'bootstrap.servers': 'kafka-1:19092,kafka-2:19093',
    'group.id': 'test-consumer-group',
    'default.topic.config': {
        'auto.offset.reset': 'largest'
    }
})

c.subscribe([topicName])

In [58]:
#single run for testing
# todo fix avro schema
def SingleRun():
    go_on = True

    while go_on:
        msg = c.poll(1.0)

        if msg is None:
            continue
        if msg.error():
            if msg.error().code() == KafkaError._PARTITION_EOF:
                continue
            else:
                print(msg.error())
                break

       # print('Received message: {}'.format(msg.value().decode('utf-8')))
        WritePM10Avro(msg.value().decode('utf-8'))
        go_on = False
        if msg.value().decode('utf-8') == "STOP":
            go_on = False

    c.close()

SingleRun()

AvroTypeException: The datum {'_id': '593fc68947df4f001113f9eb', 'createdAt': '2017-06-13T11:03:37.041Z', 'updatedAt': '2020-05-18T15:35:47.644Z', 'name': 'Freiburg im Breisgau', 'grouptag': 'XK', 'exposure': 'outdoor', 'model': 'luftdaten_sds011_dht22', 'sensors': [{'title': 'PM10', 'unit': 'µg/m³', 'sensorType': 'SDS 011', 'icon': 'osem-cloud', '_id': '593fc68947df4f001113f9ec', 'lastMeasurement': {'value': '4.05', 'createdAt': '2020-05-18T15:35:47.629Z'}}, {'title': 'PM2.5', 'unit': 'µg/m³', 'sensorType': 'SDS 011', 'icon': 'osem-cloud', '_id': '593fc68947df4f001113f9ed', 'lastMeasurement': {'value': '3.85', 'createdAt': '2020-05-18T15:35:47.629Z'}}, {'title': 'Temperatur', 'unit': '°C', 'sensorType': 'DHT22', 'icon': 'osem-thermometer', '_id': '593fc68947df4f001113f9ee', 'lastMeasurement': {'value': '25.10', 'createdAt': '2020-05-18T15:35:47.629Z'}}, {'title': 'rel. Luftfeuchte', 'unit': '%', 'sensorType': 'DHT22', 'icon': 'osem-humidity', '_id': '593fc68947df4f001113f9ef', 'lastMe

In [ ]:


#go_on = True
#
#while go_on:
#    msg = c.poll(1.0)
#
#    if msg is None:
#        continue
#    if msg.error():
#        if msg.error().code() == KafkaError._PARTITION_EOF:
#            continue
#        else:
#            print(msg.error())
#            break
#
#   # print('Received message: {}'.format(msg.value().decode('utf-8')))
#    WritePM10(msg.value().decode('utf-8'))
#    if msg.value().decode('utf-8') == "STOP":
#        go_on = False
#        
#c.close()

In [ ]:
go_on = False